## 08: Extra tree clasifier con Pycaret

In [1]:
!pip install scikit-optimize
!pip install optuna
!pip install pycaret
from pycaret.classification import *
import pandas as pd                                             #Importar libreria pandas para leer archivo parquet
pd.set_option('display.float_format', '{:.2f}'.format)          #Formatro decimal para el display de los datos
from scipy.stats import skew, kurtosis, chi2_contingency          #Importar libreria scipy para calcular skew y kurtosis
import seaborn as sns                                           #Importar libreria seaborn para graficar
from sklearn.model_selection import train_test_split, RandomizedSearchCV            #Importar libreria sklearn para separar datos en train y test
from sklearn.dummy import DummyClassifier                       #Importar libreria sklearn para crear clasificador dummy
from sklearn.metrics import roc_auc_score, roc_curve, auc       #Importar libreria sklearn para calcular metricas de clasificacion binaria
import matplotlib.pyplot as plt                                 #Importar libreria matplotlib para graficar
import numpy as np                                              #Importar libreria numpy para calculos matematicos
from sklearn.pipeline import Pipeline                           #Importar libreria sklearn para crear pipelines
from sklearn.base import BaseEstimator, TransformerMixin        #La clase ColumnSelector hereda de BaseEstimator y TransformerMixin, lo que permite integrarse fácilmente en un pipeline de scikit-learn
from sklearn.impute import SimpleImputer                        #Importar libreria sklearn para imputar valores faltantes
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, MinMaxScaler, RobustScaler #Importar libreria sklearn para escalar y codificar variables
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression             #Importar libreria sklearn para crear modelo de regresion logistica
from sklearn.svm import SVC                                     #Importar libreria sklearn para crear modelo de SVM
from skopt import BayesSearchCV                                 #Importar libreria skopt para optimizar hiperparametros
from skopt.space import Real, Integer                           #Importar libreria skopt para definir espacio de busqueda de hiperparametros
import optuna                                                   #Importar libreria optuna para optimizar hiperparametros
from sklearn.ensemble import RandomForestClassifier             #Importar libreria sklearn para crear modelo de Random Forest
from sklearn.model_selection import train_test_split            #
from sklearn.ensemble import RandomForestClassifier             #
from sklearn.neural_network import MLPClassifier                #Importar libreria sklearn para crear modelo de redes neuronales

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━

Dado los múltiplques inconvenientes que se obtuvieron con la optimización del SVC, al construirlo manualmente, probaremos con Pycaret

In [2]:
df_train= pd.read_parquet ("df_train.parquet")
print(df_train.shape)
train_dataset, test_data_set = train_test_split(df_train, test_size=0.01, random_state=844)
print(train_dataset.shape)
print(test_data_set.shape)


(23494, 46)
(23259, 46)
(235, 46)


In [3]:
X_test, y_test = test_data_set.drop(columns="Target"), test_data_set["Target"]
print(X_test.shape)
print(y_test.shape)

(235, 45)
(235,)


In [4]:
clf = setup(data=train_dataset, target="Target", fold=4, session_id=844)

,Description,Value
0,Session id,844
1,Target,Target
2,Target type,Binary
3,Original data shape,"(23259, 46)"
4,Transformed data shape,"(23259, 96)"
5,Transformed train set shape,"(16281, 96)"
6,Transformed test set shape,"(6978, 96)"
7,Numeric features,24
8,Categorical features,21
9,Rows with missing values,100.0%


In [5]:
model = create_model('et')


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8403,0.7660,0.4828,0.9180,0.6328,0.5430,0.5888
1,0.8300,0.7464,0.4440,0.9164,0.5981,0.5063,0.5598
2,0.8366,0.7762,0.4681,0.9188,0.6202,0.5297,0.5787
3,0.8452,0.7764,0.5060,0.9115,0.6508,0.5616,0.6017
Mean,0.8380,0.7662,0.4752,0.9162,0.6255,0.5352,0.5822
Std,0.0056,0.0122,0.0225,0.0028,0.0192,0.0201,0.0153


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
tuned_model = tune_model(model, optimize='AUC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5647,0.6028,0.5983,0.3470,0.4392,0.1229,0.1351
1,0.5767,0.6075,0.5957,0.3553,0.4451,0.1369,0.1489
2,0.5865,0.6203,0.6276,0.3679,0.4638,0.1631,0.1785
3,0.5794,0.6152,0.6034,0.3586,0.4499,0.1437,0.1565
Mean,0.5768,0.6114,0.6063,0.3572,0.4495,0.1416,0.1548
Std,0.0078,0.0068,0.0126,0.0075,0.0091,0.0145,0.0157


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 4 folds for each of 10 candidates, totalling 40 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [7]:
evaluate_model(tuned_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [8]:
print(tuned_model)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     monotonic_cst=None, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=844, verbose=0,
                     warm_start=False)


In [9]:
predictions = predict_model(tuned_model , data=X_test, raw_score=True)
predictions.head(5)

,MEDICAMENTOS,MEDICINA ESPECIALIZADA,MEDICINA GENERAL,Cant_gr_flia,Cant_riesgos_flia_mean,min_Tiempo_CP_Fliar,Cant_Fliar_CP,psa_max_gr_flia,psa_min_gr_flia,Cant_Fliar_riesgos,...,CEREBRAL_FAMILIAR,ENFERMEDAD_RENAL,ENFERMEDAD_RENAL_FAMILIAR,OTROS_ANTECEDENTES_VASCULARES,Pendiente,Intercepto,Promedio_costo,prediction_label,prediction_score_0,prediction_score_1
ID,,,,,,,,,,,,,,,,,,,,,
26033,0.00,0.00,1.00,2,0.00,NaN,0.00,NaN,NaN,0.00,...,0,0,0,0,11974.29,-18626.67,139700.00,1,0.00,1.00
25149,1.00,0.00,1.00,1,0.00,NaN,0.00,NaN,NaN,0.00,...,0,0,0,0,712.71,1663.00,24945.00,0,0.98,0.02
18229,15.00,5.00,0.00,1,2.00,NaN,0.00,NaN,NaN,1.00,...,0,0,0,0,-36127.09,299454.81,207612.00,0,0.86,0.14
21010,3.00,0.00,1.00,1,1.00,NaN,0.00,NaN,NaN,1.00,...,0,0,0,0,-684.69,6390.40,23964.00,0,0.73,0.27
412,2.00,0.00,2.00,3,0.00,NaN,0.00,NaN,NaN,0.00,...,2,2,2,2,11021.14,-17144.00,128580.00,0,0.76,0.24


In [10]:
y_pred = predictions['prediction_label']
roc_auc = roc_auc_score(y_test, y_pred)
print(roc_auc)

0.8106370543541789


Finalmente observamos que el modelo arroja resultados acpetables, haremos un submission

In [11]:
X_test_submission = pd.read_parquet ("df_test.parquet")
predictions = predict_model(tuned_model , data=X_test_submission, raw_score=True)
predictions.head(10)

,MEDICAMENTOS,MEDICINA ESPECIALIZADA,MEDICINA GENERAL,Cant_gr_flia,Cant_riesgos_flia_mean,min_Tiempo_CP_Fliar,Cant_Fliar_CP,psa_max_gr_flia,psa_min_gr_flia,Cant_Fliar_riesgos,...,CEREBRAL_FAMILIAR,ENFERMEDAD_RENAL,ENFERMEDAD_RENAL_FAMILIAR,OTROS_ANTECEDENTES_VASCULARES,Pendiente,Intercepto,Promedio_costo,prediction_label,prediction_score_0,prediction_score_1
ID,,,,,,,,,,,,,,,,,,,,,
12596,2.00,4.00,2.00,2,0.00,NaN,0.00,NaN,NaN,0.00,...,0,0,0,0,5421179.50,-12566170.00,9611937.00,0,0.69,0.31
29326,4.00,0.00,1.00,1,0.00,NaN,0.00,NaN,NaN,0.00,...,0,0,0,0,-9504.34,59977.87,80138.00,0,0.81,0.19
24338,0.00,0.00,1.00,2,0.00,NaN,0.00,NaN,NaN,0.00,...,0,0,0,0,2477.14,-5780.00,17340.00,0,0.70,0.30
11167,1.00,0.00,0.00,1,0.00,NaN,0.00,NaN,NaN,0.00,...,0,1,0,0,3597.14,-8393.33,25180.00,0,0.74,0.26
22074,36.00,0.00,1.00,3,0.67,NaN,0.00,NaN,NaN,1.00,...,0,0,0,0,-9042.51,630988.12,599339.31,0,0.82,0.18
21192,1.00,1.00,1.00,1,0.00,NaN,0.00,NaN,NaN,0.00,...,0,0,0,0,8622.00,-16475.33,41105.00,0,0.92,0.08
11723,1.00,2.00,0.00,1,1.00,NaN,0.00,NaN,NaN,1.00,...,0,0,0,0,18521.29,-11681.67,159428.50,0,0.84,0.16
8231,7.00,1.00,2.00,2,0.00,NaN,0.00,NaN,NaN,0.00,...,2,2,2,2,3905.71,15740.00,58820.00,0,0.88,0.12
16389,2.00,1.00,1.00,1,0.00,NaN,0.00,NaN,NaN,0.00,...,0,0,0,0,5192.71,-5465.67,38126.50,0,0.84,0.16


In [12]:
y_pred_prob_class_1 = predictions['prediction_score_1']
# Crear un DataFrame con la columna de predicciones
submission_df = pd.DataFrame({
    'ID': predictions.index,  # Suponiendo que hay un índice que puedes usar como ID
    'Target': y_pred_prob_class_1  # Cambia el nombre si es diferente
})

submission_df.to_csv('sample_submission_ET.csv', index=False)